# Webscrape BMCC Courses From Catalog

## Steps

### Webscraping
Here I will use a combination of the **Selenium** and **BeautifulSoup** packages in python. First, I will select a specific department on the BMCC website. Then, I will obtain a list of links to every single course using Selenium. Afterwards, I will iterate through every link using BeautifulSoup in order to obtain the information for each course. I then use Selenium again to obtain course prerequistes that could not be obtained using BeautifulSoup. Finally, I will combine all of the course information into a single dataframe and export it as a csv file. 

### Connecting to AI Agent
I will then upload that csv file onto **Supabase** and connect the Supabase node to the ai agent in **n8n**. The AI Agent will refer to the Supabase node in order to answer student questions.

## Importing Packages

In [1]:
import pandas as pd 
import numpy as np

# for combining the course data
from functools import reduce

# for webscraping the course page
import requests
from bs4 import BeautifulSoup
from io import StringIO
import re

# for webscraping the list of progam and course links
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time

## Getting The Link to Every Course in the Department

This Jupyter Notebook will only contain a ***single*** department ([ACCOUNTING]('https://bmcc.catalog.cuny.edu/courses?department=ACCT-BMC&page=1&cq=')), but the code can be used for all departments in BMCC's catalog. Since most CUNYs have a similar catalog set up, this code could also be used to webscrape the catalogs of other CUNY schools.

In [2]:
driver = webdriver.Chrome(options = Options())

# in this example we will only look at the ACCOUNTING department.
url = 'https://bmcc.catalog.cuny.edu/courses?department=ACCT-BMC&page=1&cq='

driver.get(url)

# 'links' will contain our list of course links
links = []

# function to get the links
def get_info():
    driver.implicitly_wait(2)

    # the courses are inside of a table
    # to find the table body
    tbody = driver.find_element(By.XPATH, '/html/body/div/div/div/div/div[2]/div/main/section/div[2]/div/table/tbody')

    # going through all the rows in our table to webscrape the links for each course listed
    for tr in tbody.find_elements(By.XPATH, '//tr'):
        driver.implicitly_wait(2)
        link = [item.get_attribute('href') for item in tr.find_elements(By.CSS_SELECTOR, 'a')]
        # adding each link into our list of links
        links.append(link)


driver.implicitly_wait(5)

# function to click through each page and then get links
for i in range(15):
    try:
        get_info()
        button = driver.find_element(By.XPATH, "//button[@aria-label='Next page']")
        button.click()

        
        # Wait 2 seconds before the next click
        time.sleep(15) 
    except Exception as e:
        break
        
driver.quit()

In [3]:
cl_links = list(filter(None, links))

# will hold links to each course, that we can then use to get each course's information
clean_links = []

for links in cl_links:
    for link in links:
        clean_links.append(link)

## Getting Course Info

In [4]:
courses_list = []

for link in clean_links:

    url = link
    
    r= requests.get(url)
    
    soup = BeautifulSoup(r.text, "html.parser")
    
    # course information is found in different sections, 'divisions' contains a list of eachs section's id
    divisions = ['NX3AU', '1LxJi', 'ElFBC']
    
    # 'Labels' will contain our list of field labels
    Labels = []
    # 'Values' will contain our list of field values
    Values = []
    
    for div in divisions:
        sections = soup.find(id = div)
        # not all courses have the same number of sections, this will skip a section that is not present on the course webpage
        if sections == None:
            pass
        else:
            # to find all the labels in our sections
            labels = sections.find_all('h3', class_='field-label')
            for label in labels:
                # to add all the labels into our list of labels
                Labels.append(label.text)
            # to find all the values in our sections
            values = sections.find_all('div', class_='field-value')
            for value in values:
                # to add all the values into our list of values
                Values.append(value.text)
    
    
    # 'all_values' keeps the labels and values of each course together
    all_values = [[Labels, Values]]

    # this will add each course's labels and values into a list
    for a in all_values:
        courses_list.append(a)

In [5]:
# 'df_list' will contain a list of dataframes
df_list = []

# this will create a dataframe using the labels list and values list of each course
for cl in courses_list:
    label = cl[0]
    value = cl[1]
    dict = {'Labels': label, 'Values': value} 
    df = pd.DataFrame(dict)
    # this will rename each column holding course information to that course's name
    df.rename(columns = {'Values': value[0] + " " + value[1]}, inplace=True)
    df_list.append(df)

In [6]:
# to see the first df in our list
df_list[0]

,Labels,ACC 231
0,Subject area,ACC
1,Catalog Number,231
2,Course Title,Accounting Applications on Microcomputers
3,Department(s),\n Accounting\n
4,Description,This course provides accounting students with ...
5,Typically Offered,\n All Terms\n
6,Academic Career,Undergraduate
7,Liberal Arts,No
8,Minimum Units,3
9,Maximum Units,3


### Merging the DFs

In [7]:
# doing an outer join to combine the course info, using our list of dfs
df_merged = reduce(lambda left,right: pd.merge(left,right,on=['Labels'], how='outer'), df_list)

# transposing the df
bmcc_courses = df_merged.T

# renaming the columns, using the 'Labels' row
bmcc_courses.rename(columns=bmcc_courses.iloc[0], inplace = True)

# dropping the 'Labels' row, now that we've edited our header
bmcc_courses.drop(bmcc_courses.index[0], inplace = True)

# add name to first column
bmcc_courses.index.name = 'Course'

bmcc_courses

,Subject area,Catalog Number,Course Title,Department(s),Description,Typically Offered,Academic Career,Liberal Arts,Minimum Units,Maximum Units,Academic Progress Units,Repeat For Credit,
Course,,,,,,,,,,,,,
ACC 231,ACC,231,Accounting Applications on Microcomputers,\n Accounting\n,This course provides accounting students with ...,\n All Terms\n,Undergraduate,No,3,3,3,No,022063
ACC 331,ACC,331,Accounting Information Systems,\n Accounting\n,The course provides accounting students with t...,\n All Terms\n,Undergraduate,No,3,3,3,No,013557
ACC 331H,ACC,331H,Accounting Information Systems (Honors),\n Accounting\n,The course provides accounting students with t...,\n All Terms\n,Undergraduate,No,3,3,3,No,028420
ACC 122,ACC,122,Accounting Principles I,\n Accounting\n,The course covers the fundamental principles o...,\n All Terms\n,Undergraduate,No,3,3,3,No,013559
ACC 222,ACC,222,Accounting Principles II,\n Accounting\n,This continuation of Accounting I progresses f...,\n All Terms\n,Undergraduate,No,3,3,3,No,028895
ACC 350,ACC,350,Cost Accounting I,\n Accounting\n,"Emphasis is placed on the conceptual, analytic...",\n All Terms\n,Undergraduate,No,3,3,3,No,022063
ACC 451,ACC,451,Cost Accounting II,\n Accounting\n,The uses of cost accounting concepts and metho...,"\n Fall, Spring\n",Undergraduate,No,3,3,3,No,031894
ACC 370,ACC,370,Forensic Accounting & Fraud Analysis,\n Accounting\n,The course is an introduction to Forensic Acco...,\n All Terms\n,Undergraduate,No,3,3,3,No,016345
ACC 360,ACC,360,Government and Not-For-Profit Accounting,\n Accounting\n,This course introduces the theory and concepts...,\n All Terms\n,Undergraduate,No,3,3,3,No,016345


### Getting Requisites
* prereqs were missing from our previous table, they showed up as a 6 digit number

In [8]:
reqs_all = []

driver = webdriver.Chrome()

for course in clean_links:
    driver.get(course)
    
    subject = driver.find_element(By.XPATH, '//*[@id="NX3AU"]/div/div[2]/div[1]/div').text
    number = driver.find_element(By.XPATH, '//*[@id="NX3AU"]/div/div[2]/div[2]/div').text
    course = subject + ' ' + number
    
    reqs = []

    try: 
        time.sleep(2)
        # using driver.implicitly_wait(10) or expected_conditions is too fast and sometimes the prereqs do not fully load before scraping
        driver.find_element(By.XPATH, '//*[@id="ElFBC"]/div/div/div/div').text
        pre_req = driver.find_element(By.XPATH, '//*[@id="ElFBC"]/div/div/div/div').text
        requisite = course + ' ' + pre_req
        reqs.append(requisite)
    except NoSuchElementException:
        pass

    for req in reqs:
        reqs_all.append(req)

driver.quit()

In [9]:
requirements = pd.DataFrame(reqs_all, columns = ['PreReqs'])

#### Cleaning Requisites

In [10]:
requirements_clean = pd.DataFrame(columns = ['Course', 'Requisite'])

# to split the course name and prereq into two columns
requirements_clean['Course'] = [r[:8] for r in requirements['PreReqs']]
requirements_clean['Requisite'] = [r[8:] for r in requirements['PreReqs']]

requirements_clean['Course'] = requirements_clean['Course'].str.strip()
requirements_clean['Requisite'] = requirements_clean['Requisite'].str.strip()
requirements_clean['Course'] = requirements_clean['Course'].str.replace('.', '', regex=False)

In [11]:
# to remove all text after the second space in the Courses column
def remove_text_after_second_space(text):
    parts = text.split(maxsplit=2)
    if len(parts) > 2:
        return ' '.join(parts[:2])
    return text

courses_CLEAN = []

for course in requirements_clean['Course']:
    cl_course = remove_text_after_second_space(course)
    courses_CLEAN.append(cl_course)

requirements_clean['Course'] = courses_CLEAN

In [12]:
requirements_clean

,Course,Requisite
0,ACC 231,Prerequisite: ACC 122 Students must also meet ...
1,ACC 331,Prerequisite: English proficiency index of 55+...
2,ACC 331H,This is an honors section. Students must have ...
3,ACC 122,Prerequisite: ESL 54 or higher and Math Profic...
4,ACC 222,Prerequisite: ACC 122 Students must also meet ...
5,ACC 350,Prerequisite: ACC 122 Students must also meet ...
6,ACC 451,Prerequisite: ACC 350Students must also meet t...
7,ACC 370,Prerequisite: ACC 122Students must have an Eng...
8,ACC 360,Prerequisite: ACC 122Students must have an Eng...
9,ACC 330,Prerequisite: ACC 222 Students must have an En...


### Merging the Information

In [13]:
bmcc_courses2 = pd.merge(bmcc_courses, requirements_clean, on= ['Course'], how='outer')

# adding a name to the column without name
bmcc_courses2.rename(columns={'': 'remove'}, inplace= True)

## Creating new DF with main course info

In [14]:
# creating the new DF with necessary information 
bmcc_courses_clean = bmcc_courses2.filter(['Course', 'Course Title', 'Department(s)', 'Description', 'Typically Offered',  
                                           'Liberal Arts', 'Maximum Units', 'Repeat For Credit' , 
                                           'Requirement Designation', 'Requisite', 'Academic Career'], axis = 1)

# CLEANING
# removing extra strings
bmcc_courses_clean['Department(s)'] = bmcc_courses_clean['Department(s)'].str.replace('\n', '', regex=False)
bmcc_courses_clean['Typically Offered'] = bmcc_courses_clean['Typically Offered'].str.replace('\n', '', regex=False)
bmcc_courses_clean['Requirement Designation'] = np.nan
#bmcc_courses_clean['Requirement Designation'] = bmcc_courses_clean['Requirement Designation'].str.replace('\n', '', regex=False)

# removing extra strings spaces
bmcc_courses_clean['Department(s)'] = bmcc_courses_clean['Department(s)'].str.strip()
bmcc_courses_clean['Typically Offered'] = bmcc_courses_clean['Typically Offered'].str.strip()

# renaming credits column
bmcc_courses_clean.rename(columns={'Maximum Units': 'Credits'}, inplace= True)

# removing duplicates
bmcc_courses_clean = bmcc_courses_clean.drop_duplicates(subset=['Course', 'Course Title'], keep='first')

### Reordering Columns

In [15]:
new_column_order = ['Course', 'Course Title', 'Department(s)', 'Description', 'Credits', 'Repeat For Credit', 
                    'Requisite', 'Requirement Designation','Typically Offered',  'Liberal Arts', 'Academic Career']

bmcc_courses_clean = bmcc_courses_clean[new_column_order]    

bmcc_courses_clean

,Course,Course Title,Department(s),Description,Credits,Repeat For Credit,Requisite,Requirement Designation,Typically Offered,Liberal Arts,Academic Career
0,ACC 231,Accounting Applications on Microcomputers,Accounting,This course provides accounting students with ...,3,No,Prerequisite: ACC 122 Students must also meet ...,NaN,All Terms,No,Undergraduate
1,ACC 331,Accounting Information Systems,Accounting,The course provides accounting students with t...,3,No,Prerequisite: English proficiency index of 55+...,NaN,All Terms,No,Undergraduate
2,ACC 331H,Accounting Information Systems (Honors),Accounting,The course provides accounting students with t...,3,No,This is an honors section. Students must have ...,NaN,All Terms,No,Undergraduate
3,ACC 122,Accounting Principles I,Accounting,The course covers the fundamental principles o...,3,No,Prerequisite: ESL 54 or higher and Math Profic...,NaN,All Terms,No,Undergraduate
4,ACC 222,Accounting Principles II,Accounting,This continuation of Accounting I progresses f...,3,No,Prerequisite: ACC 122 Students must also meet ...,NaN,All Terms,No,Undergraduate
5,ACC 350,Cost Accounting I,Accounting,"Emphasis is placed on the conceptual, analytic...",3,No,Prerequisite: ACC 122 Students must also meet ...,NaN,All Terms,No,Undergraduate
6,ACC 451,Cost Accounting II,Accounting,The uses of cost accounting concepts and metho...,3,No,Prerequisite: ACC 350Students must also meet t...,NaN,"Fall, Spring",No,Undergraduate
7,ACC 370,Forensic Accounting & Fraud Analysis,Accounting,The course is an introduction to Forensic Acco...,3,No,Prerequisite: ACC 122Students must have an Eng...,NaN,All Terms,No,Undergraduate
8,ACC 360,Government and Not-For-Profit Accounting,Accounting,This course introduces the theory and concepts...,3,No,Prerequisite: ACC 122Students must have an Eng...,NaN,All Terms,No,Undergraduate
9,ACC 330,Intermediate Accounting I,Accounting,The course begins with a review of the account...,3,No,Prerequisite: ACC 222 Students must have an En...,NaN,All Terms,No,Undergraduate


## Exporting the Data

* change file name based on department

In [16]:
bmcc_courses_clean.to_csv('accounting.csv', index=False)